In [1]:
# 패키지 로드
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from collections import defaultdict
import os, random

from scipy import sparse
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
from torch.nn.init import normal_
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F

from sklearn.feature_selection import chi2, f_classif, SelectKBest

#import plotnine
#from plotnine import *

In [2]:
# 데이터 불러오기 
df = pd.read_csv('/Data2/hm22/New/sample.csv', encoding='utf-8')
df = df.drop(['Unnamed: 0'], axis = 1)

In [3]:
df

,profile_id,album_id,ss_id,log_time_x,log_time_y,diff,diff_ss,diff_ox,watch_time,total_time,...,search_ox,prefer,sex,age,pr_interest_keyword_cd_1,pr_interest_keyword_cd_2,pr_interest_keyword_cd_3,ch_interest_keyword_cd_1,ch_interest_keyword_cd_2,ch_interest_keyword_cd_3
0,3,15.0,2022-03-01 11:56:53,2022-03-01 11:57:19,2022-03-01 11:58:05,0 days 00:00:46,46.0,True,46.0,46.0,...,0.0,4.0,F,5,P02,P04,P07,K01,K03,K04
1,3,16.0,2022-03-01 11:56:53,2022-03-01 11:58:09,2022-03-01 11:59:54,0 days 00:01:45,105.0,False,104.0,105.0,...,0.0,4.0,F,5,P02,P04,P07,K01,K03,K04
2,3,17.0,2022-03-01 11:56:53,2022-03-01 11:59:58,2022-03-01 12:01:14,0 days 00:01:16,76.0,True,76.0,76.0,...,0.0,4.0,F,5,P02,P04,P07,K01,K03,K04
3,3,18.0,2022-03-01 11:56:53,2022-03-01 12:01:18,2022-03-01 12:02:26,0 days 00:01:08,68.0,False,67.0,68.0,...,0.0,4.0,F,5,P02,P04,P07,K01,K03,K04
4,3,19.0,2022-03-01 11:56:53,2022-03-01 12:02:29,2022-03-01 12:04:00,0 days 00:01:31,91.0,False,90.0,90.0,...,0.0,4.0,F,5,P02,P04,P07,K01,K03,K04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1914989,33032,381.0,2022-04-27 15:50:09,2022-04-27 15:56:42,2022-04-27 15:56:53,0 days 00:00:11,11.0,False,463.0,464.0,...,0.0,4.0,F,5,P03,P05,P07,K01,K05,K09
1914990,33032,381.0,2022-04-27 15:50:09,2022-04-27 15:56:42,2022-04-27 15:56:09,-1 days +23:59:27,-33.0,False,462.0,463.0,...,0.0,4.0,F,5,P03,P05,P07,K01,K05,K09
1914991,33032,375.0,2022-04-27 15:50:09,2022-04-27 15:55:59,2022-04-27 15:55:26,-1 days +23:59:27,-33.0,False,658.0,659.0,...,0.0,4.0,F,5,P03,P05,P07,K01,K05,K09
1914992,33032,125.0,2022-04-27 15:58:39,2022-04-27 15:58:10,2022-04-27 15:58:26,0 days 00:00:16,16.0,False,10.0,520.0,...,0.0,1.0,F,5,P03,P05,P07,K01,K05,K09


In [4]:
df.columns

Index(['profile_id', 'album_id', 'ss_id', 'log_time_x', 'log_time_y', 'diff',
       'diff_ss', 'diff_ox', 'watch_time', 'total_time', 'continuous_play_x',
       'continuous_play_y', 'act_target_dtl_x', 'act_target_dtl',
       'short_trailer', 'payment', 'time_slot', 'search_ox', 'prefer', 'sex',
       'age', 'pr_interest_keyword_cd_1', 'pr_interest_keyword_cd_2',
       'pr_interest_keyword_cd_3', 'ch_interest_keyword_cd_1',
       'ch_interest_keyword_cd_2', 'ch_interest_keyword_cd_3'],
      dtype='object')

In [5]:
# 시청 시작 데이터에서 연속재생여부 Y -> 1: 연속재생, N -> 5: 미정 으로 변환
df.loc[(df['continuous_play_x'] == 'Y'), 'continuous_play_x'] = 1
df.loc[(df['continuous_play_x'] == 'N'), 'continuous_play_x'] = 0

In [6]:
df['short_trailer'] = df['short_trailer'].apply(lambda x: 1 if x == 'Y' else 0)
df['sex'] = df['sex'].apply(lambda x: 1 if x == 'M' else 0)

In [7]:
df['act_target_dtl'] = df['act_target_dtl'].str.split('D').str[1]
df['act_target_dtl_x'] = df['act_target_dtl_x'].str.split('D').str[1]

In [8]:
# 주의 : 결측치가 없어야함
for i in ['pr_interest_keyword_cd_1', 'pr_interest_keyword_cd_2', 'pr_interest_keyword_cd_3',
          'ch_interest_keyword_cd_1', 'ch_interest_keyword_cd_2', 'ch_interest_keyword_cd_3'] : 
    enc_1 = (df.groupby(i).size()) / len(df)
    enc_1
    
    df['{}_encode'.format(i)] = df[i].apply(lambda x : enc_1[x])
    
#인코딩 전 변수는 제거 
df = df.drop(['pr_interest_keyword_cd_1', 'pr_interest_keyword_cd_2', 'pr_interest_keyword_cd_3',
                'ch_interest_keyword_cd_1', 'ch_interest_keyword_cd_2', 'ch_interest_keyword_cd_3'],axis=1)

In [9]:
X = df.loc[:, ~df.columns.isin(['album_id'])] # FEATURE DATA
y = df['album_id'] # LABEL DATA

In [14]:
X['ss_id'] = pd.to_numeric(pd.to_datetime(X['ss_id']))
X['log_time_x'] = pd.to_numeric(pd.to_datetime(X['log_time_x']))
X['log_time_y'] = pd.to_numeric(pd.to_datetime(X['log_time_y']))

X = X.drop(['diff'], axis = 1)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, test_size = 0.2)

In [16]:
## selctor 정의하기
selector = SelectKBest(score_func = f_classif, k = 15)

## 학습데이터에 fit_transform 
X_train_selected = selector.fit_transform(X_train, y_train)

## 테스트 데이터는 transform
X_test_selected = selector.transform(X_test)
X_train_selected.shape, X_test_selected.shape

((1531995, 15), (382999, 15))

In [17]:
all_names = X_train.columns
## selector.get_support()
selected_mask = selector.get_support()
## 선택된 특성(변수)들
selected_names = all_names[selected_mask]
## 선택되지 않은 특성(변수)들
unselected_names = all_names[~selected_mask]
print('Selected names: ', selected_names)
print('Unselected names: ', unselected_names)

Selected names:  Index(['ss_id', 'log_time_x', 'log_time_y', 'watch_time', 'total_time',
       'continuous_play_x', 'short_trailer', 'payment', 'prefer', 'sex', 'age',
       'pr_interest_keyword_cd_3_encode', 'ch_interest_keyword_cd_1_encode',
       'ch_interest_keyword_cd_2_encode', 'ch_interest_keyword_cd_3_encode'],
      dtype='object')
Unselected names:  Index(['profile_id', 'diff_ss', 'diff_ox', 'continuous_play_y',
       'act_target_dtl_x', 'act_target_dtl', 'time_slot', 'search_ox',
       'pr_interest_keyword_cd_1_encode', 'pr_interest_keyword_cd_2_encode'],
      dtype='object')


In [26]:
X = df.loc[:, ~df.columns.isin(['album_id'])] # FEATURE DATA
y = df['album_id'] # LABEL DATA

X['ss_id'] = pd.to_numeric(pd.to_datetime(X['ss_id']))
X['log_time_x'] = pd.to_numeric(pd.to_datetime(X['log_time_x']))
X['log_time_y'] = pd.to_numeric(pd.to_datetime(X['log_time_y']))

X = X.drop(['diff'], axis = 1)
X

,profile_id,ss_id,log_time_x,log_time_y,diff_ss,diff_ox,watch_time,total_time,continuous_play_x,continuous_play_y,...,search_ox,prefer,sex,age,pr_interest_keyword_cd_1_encode,pr_interest_keyword_cd_2_encode,pr_interest_keyword_cd_3_encode,ch_interest_keyword_cd_1_encode,ch_interest_keyword_cd_2_encode,ch_interest_keyword_cd_3_encode
0,3,1646135813000000000,1646135839000000000,1646135885000000000,46.0,True,46.0,46.0,1,1.0,...,0.0,4.0,0,5,0.215737,0.118302,0.20921,0.59852,0.156005,0.069369
1,3,1646135813000000000,1646135889000000000,1646135994000000000,105.0,False,104.0,105.0,1,1.0,...,0.0,4.0,0,5,0.215737,0.118302,0.20921,0.59852,0.156005,0.069369
2,3,1646135813000000000,1646135998000000000,1646136074000000000,76.0,True,76.0,76.0,1,1.0,...,0.0,4.0,0,5,0.215737,0.118302,0.20921,0.59852,0.156005,0.069369
3,3,1646135813000000000,1646136078000000000,1646136146000000000,68.0,False,67.0,68.0,1,1.0,...,0.0,4.0,0,5,0.215737,0.118302,0.20921,0.59852,0.156005,0.069369
4,3,1646135813000000000,1646136149000000000,1646136240000000000,91.0,False,90.0,90.0,1,0.0,...,0.0,4.0,0,5,0.215737,0.118302,0.20921,0.59852,0.156005,0.069369
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1914989,33032,1651074609000000000,1651075002000000000,1651075013000000000,11.0,False,463.0,464.0,0,1.0,...,0.0,4.0,0,5,0.174492,0.122729,0.20921,0.59852,0.073859,0.085907
1914990,33032,1651074609000000000,1651075002000000000,1651074969000000000,-33.0,False,462.0,463.0,0,1.0,...,0.0,4.0,0,5,0.174492,0.122729,0.20921,0.59852,0.073859,0.085907
1914991,33032,1651074609000000000,1651074959000000000,1651074926000000000,-33.0,False,658.0,659.0,1,1.0,...,0.0,4.0,0,5,0.174492,0.122729,0.20921,0.59852,0.073859,0.085907
1914992,33032,1651075119000000000,1651075090000000000,1651075106000000000,16.0,False,10.0,520.0,1,0.0,...,0.0,1.0,0,5,0.174492,0.122729,0.20921,0.59852,0.073859,0.085907


In [36]:
X.columns

Index(['profile_id', 'ss_id', 'log_time_x', 'log_time_y', 'diff_ss', 'diff_ox',
       'watch_time', 'total_time', 'continuous_play_x', 'continuous_play_y',
       'act_target_dtl_x', 'act_target_dtl', 'short_trailer', 'payment',
       'time_slot', 'search_ox', 'prefer', 'sex', 'age',
       'pr_interest_keyword_cd_1_encode', 'pr_interest_keyword_cd_2_encode',
       'pr_interest_keyword_cd_3_encode', 'ch_interest_keyword_cd_1_encode',
       'ch_interest_keyword_cd_2_encode', 'ch_interest_keyword_cd_3_encode'],
      dtype='object')

In [27]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, train_size = 0.8, test_size = 0.2)

In [29]:
X_train

array([[0.23685246, 0.0630171 , 0.06360981, ..., 1.        , 0.89588129,
        0.33129678],
       [0.16046505, 0.71324126, 0.71447955, ..., 0.04569749, 0.57284115,
        0.00529032],
       [0.23515698, 0.44927701, 0.44941981, ..., 0.07064765, 0.89588129,
        0.00529032],
       ...,
       [0.14647734, 0.22617649, 0.22650947, ..., 1.        , 1.        ,
        1.        ],
       [0.86036513, 0.76240937, 0.76234322, ..., 0.13764407, 0.14981015,
        0.33630075],
       [0.80986406, 0.30613602, 0.30656564, ..., 0.13764407, 0.03193103,
        0.01042753]])

In [30]:
# selctor 정의하기
selector2 = SelectKBest(score_func = chi2, k = 15)

## 학습데이터에 fit_transform 
X_train_selected2 = selector2.fit_transform(X_train, y_train)

## 테스트 데이터는 transform
X_test_selected2 = selector2.transform(X_test)
X_train_selected2.shape, X_test_selected2.shape

((1531995, 15), (382999, 15))

In [33]:
X_train = pd.DataFrame(X_train)

In [34]:
all_names = X_train.columns
## selector.get_support()
selected_mask2 = selector2.get_support()
## 선택된 특성(변수)들
selected_names2 = all_names[selected_mask2]
## 선택되지 않은 특성(변수)들
unselected_names2 = all_names[~selected_mask2]
print('Selected names: ', selected_names2)
print('Unselected names: ', unselected_names2)

Selected names:  Int64Index([0, 1, 2, 3, 5, 7, 12, 13, 15, 16, 17, 18, 22, 23, 24], dtype='int64')
Unselected names:  Int64Index([4, 6, 8, 9, 10, 11, 14, 19, 20, 21], dtype='int64')


In [37]:
X_train.columns = ['profile_id', 'ss_id', 'log_time_x', 'log_time_y', 'diff_ss', 'diff_ox',
       'watch_time', 'total_time', 'continuous_play_x', 'continuous_play_y',
       'act_target_dtl_x', 'act_target_dtl', 'short_trailer', 'payment',
       'time_slot', 'search_ox', 'prefer', 'sex', 'age',
       'pr_interest_keyword_cd_1_encode', 'pr_interest_keyword_cd_2_encode',
       'pr_interest_keyword_cd_3_encode', 'ch_interest_keyword_cd_1_encode',
       'ch_interest_keyword_cd_2_encode', 'ch_interest_keyword_cd_3_encode']

In [38]:
X_train

,profile_id,ss_id,log_time_x,log_time_y,diff_ss,diff_ox,watch_time,total_time,continuous_play_x,continuous_play_y,...,search_ox,prefer,sex,age,pr_interest_keyword_cd_1_encode,pr_interest_keyword_cd_2_encode,pr_interest_keyword_cd_3_encode,ch_interest_keyword_cd_1_encode,ch_interest_keyword_cd_2_encode,ch_interest_keyword_cd_3_encode
0,0.236852,0.063017,0.063610,0.063583,0.533260,0.0,0.019711,0.008530,1.0,0.333333,...,0.0,1.000000,0.0,0.250000,1.000000,0.481117,0.647933,1.000000,0.895881,0.331297
1,0.160465,0.713241,0.714480,0.714387,0.533220,0.0,0.003145,0.072503,1.0,0.333333,...,0.0,0.000000,1.0,0.500000,0.469751,0.045300,0.586544,0.045697,0.572841,0.005290
2,0.235157,0.449277,0.449420,0.449395,0.533286,1.0,0.012791,0.009428,1.0,0.000000,...,0.0,0.666667,1.0,0.250000,0.417664,0.481117,0.195553,0.070648,0.895881,0.005290
3,0.792395,0.303550,0.303856,0.303837,0.533285,1.0,0.011533,0.056453,1.0,0.666667,...,0.0,0.000000,0.0,0.583333,0.789539,0.481117,0.373913,0.137644,0.031931,0.240604
4,0.930516,0.682852,0.682879,0.682828,0.533268,0.0,0.063745,0.045455,1.0,0.666667,...,0.0,0.666667,1.0,0.416667,0.855160,0.553764,0.195553,0.137644,0.240704,0.005290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1531990,0.534772,0.030851,0.030983,0.030922,0.533215,0.0,0.002726,0.006846,1.0,0.666667,...,0.0,0.000000,1.0,0.250000,0.457031,0.434691,0.647933,0.000000,0.240704,0.240604
1531991,0.084501,0.917703,0.918937,0.917971,0.532154,0.0,0.019920,0.008530,1.0,0.333333,...,0.0,1.000000,0.0,0.083333,0.789539,0.640158,0.647933,1.000000,1.000000,1.000000
1531992,0.146477,0.226176,0.226509,0.226528,0.533326,0.0,0.003145,0.028620,1.0,0.333333,...,0.0,0.000000,1.0,0.166667,1.000000,0.412706,0.647933,1.000000,1.000000,1.000000
1531993,0.860365,0.762409,0.762343,0.762341,0.533335,0.0,0.054938,0.072615,1.0,0.000000,...,0.0,0.333333,1.0,0.583333,0.306803,0.045300,0.891598,0.137644,0.149810,0.336301


Unselected names :

'diff_ss', 'watch_time', 'continuous_play_x', 'continuous_play_y', 'act_target_dtl_x', 'act_target_dtl', 'time_slot', 'pr_interest_keyword_cd_1_encode', 'pr_interest_keyword_cd_2_encode',
'pr_interest_keyword_cd_3_encode' 



Unselected names:  

Index(['profile_id', 'diff_ss', 'diff_ox', 'continuous_play_y',
       'act_target_dtl_x', 'act_target_dtl', 'time_slot', 'search_ox',
       'pr_interest_keyword_cd_1_encode', 'pr_interest_keyword_cd_2_encode'],
      dtype='object')